Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local-advanced.png)

# Register model and deploy locally with advanced usages

This example shows how to deploy a web service in step-by-step fashion:

 1. Register model
 2. Deploy the image as a web service in a local Docker container.
 3. Quickly test changes to your entry script by reloading the local service.
 4. Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create trained model

For this example, we will train a small model on scikit-learn's [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset). 

In [ ]:
import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

dataset_x, dataset_y = load_diabetes(return_X_y=True)

sk_model = Ridge().fit(dataset_x, dataset_y)

joblib.dump(sk_model, "sklearn_regression_model.pkl")

## Register Model

You can add tags and descriptions to your models. we are using `sklearn_regression_model.pkl` file in the current directory as a model with the name `sklearn_regression_model` in the workspace.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model, framework, category, target customer etc. Note that tags must be alphanumeric.

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="sklearn_regression_model.pkl",
                       model_name="sklearn_regression_model",
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Ridge regression model to predict diabetes",
                       workspace=ws)

Manage your dependencies in a folder

In [ ]:
import os

source_directory = "source_directory"

os.makedirs(source_directory, exist_ok=True)
os.makedirs(os.path.join(source_directory, "x/y"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "env"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "dockerstep"), exist_ok=True)

Show `score.py`. 

In [ ]:
%%writefile source_directory/x/y/score.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION)
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

    global name
    # Note here, the entire source directory from inference config gets added into image.
    # Below is an example of how you can use any extra files in image.
    with open('./source_directory/extradata.json') as json_file:
        data = json.load(json_file)
        name = data["people"][0]["name"]

input_sample = np.array([[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "Hello " + name + " here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

In [ ]:
%%writefile source_directory/extradata.json
{
    "people": [
        {
            "website": "microsoft.com", 
            "from": "Seattle", 
            "name": "Mrudula"
        }
    ]
}

## Create Inference Configuration

 - file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
 - env.docker.base_dockerfile: any extra steps you want to inject into docker file
 - source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - entry_script: contains logic specific to initializing your model and running predictions

In [ ]:
import sklearn

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


myenv = Environment('myenv')
myenv.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
myenv.python.conda_dependencies.add_pip_package("joblib")
myenv.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

# explicitly set base_image to None when setting base_dockerfile
myenv.docker.base_image = None
myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
myenv.inferencing_stack_version = "latest"

inference_config = InferenceConfig(source_directory=source_directory,
                                   entry_script="x/y/score.py",
                                   environment=myenv)

## Deploy Model as a Local Docker Web Service

*Make sure you have Docker installed and running.*

Note that the service creation can take few minutes.

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

    # PowerShell command to switch to Linux engine
    & 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [ ]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

In [ ]:
print('Local service port: {}'.format(local_service.port))

## Check Status and Get Container Logs


In [ ]:
print(local_service.get_logs())

## Test Web Service

Call the web service with some input data to get a prediction.

In [ ]:
import json

sample_input = json.dumps({
    'data': dataset_x[0:2].tolist()
})

print(local_service.run(sample_input))

## Reload Service

You can update your score.py file and then call `reload()` to quickly restart the service. This will only reload your execution script and dependency files, it will not rebuild the underlying Docker image. As a result, `reload()` is fast, but if you do need to rebuild the image -- to add a new Conda or pip package, for instance -- you will have to call `update()`, instead (see below).

In [ ]:
%%writefile source_directory/x/y/score.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

    global name, from_location
    # Note here, the entire source directory from inference config gets added into image.
    # Below is an example of how you can use any extra files in image.
    with open('source_directory/extradata.json') as json_file:  
        data = json.load(json_file)
        name = data["people"][0]["name"]
        from_location = data["people"][0]["from"]

input_sample = np.array([[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "Hello " + name + " from " + from_location + " here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

In [ ]:
local_service.reload()
print("--------------------------------------------------------------")

# After calling reload(), run() will return the updated message.
local_service.run(sample_input)

## Update Service

If you want to change your model(s), Conda dependencies, or deployment configuration, call `update()` to rebuild the Docker image.

```python

local_service.update(models=[SomeOtherModelObject],
                     deployment_config=local_config,
                     inference_config=inference_config)
```

## Delete Service

In [ ]:
local_service.delete()